In [16]:
#load the environment + define the path
%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import os
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import pickle



# Get the list of recordings
info = mfun.analysis()
# display the detected session
print( "Total Session: " +  str(info.recordingList .shape[0]))
#info.recordingList.head()

# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')

info.recordingList

Takes <30s. However, it might take >1 mins or so if network is busy...
Computer: Candela Windows
Total Session: 3


c:\Users\Lak Lab\Documents\Github\sideBiasLateralisation\main_funcs.py:145: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-04-07_CMN001_1\' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.recordingList.loc[ind,'analysispathname'] = analysispathname +'\\'


Matlab engine is set correctly.


In [18]:
# Filter for a particular expRef from the info list
specific_expRef = '2025-04-09_1_CMN001'

# Recreate info object with the specific expRef
info.recordingList = info.recordingList[info.recordingList['sessionName'] == specific_expRef].reset_index(drop=True)
info.recordingList

,animalID,recordingDate,recordingID,sessionName,learningData,twoP,path,sessionNameWithPath,analysispathname,filepathname
0,CMN001,2025-04-09,1,2025-04-09_1_CMN001,False,True,Z:/CMN001\2025-04-09,Z:/CMN001\2025-04-09\1\2025-04-09_1_CMN001_Blo...,C:/Users/Lak Lab/Documents/Github/sideBiasLate...,Z:/CMN001\2025-04-09\1


In [ ]:
# create + save beh fig

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    session = str(info.recordingList.sessionName[0])
    save_path = info.recordingList.path[0]  # Get the path where to save
    save_file = os.path.join(save_path, f"{session}_behavior_plot.png")

    # Check if file already exists
    if os.path.exists(save_file):
        print(f"File already exists for session {session}, skipping visualization")
        print(f"Existing file: {save_file}")
    else:
        try:
            # Create figure and get handle
            eng.visualiseTrainingGrating2AFC(session, 'Grating2AFC_noTimeline', [], nargout=0)
            
            # Get the current figure handle
            fig = eng.gcf()
            
            # Save the figure
            eng.saveas(fig, save_file, nargout=0)
            eng.close(fig, nargout=0)  # Close the figure
            
            print(f"Visualization completed and saved for session {session}")
            print(f"Saved to: {save_file}")
        except Exception as e:
            print(f"Error when running visualiseTrainingGrating2AFC: {str(e)}")

File already exists for session 2025-05-09_1_MBL015, skipping visualization
Existing file: Z:/MBL015\2025-05-09\2025-05-09_1_MBL015_behavior_plot.png


In [ ]:
# # to check the paq file details - change filenamePAQ
# import paq2py as paq_reader
# import utils_funcs as utils

# filenamePAQ =  'Y:\\OFZ011\\2023-07-07\\TwoP\\2023-07-07_OFZ011_paq_001.paq'
# savepathname = info.recordingList()
# paq_data = paq_reader.paq_read( file_path=filenamePAQ, plot=True, save_path=savepathname) 
# frame_clock = utils.paq_data (paqData, 'reward', threshold_ttl=True, plot=False)

In [3]:
# Check CSV files 
# checkOnly needs to be False to create new files with MATLAB GENERIC CODE: GetBehavData.mat
checkOnly = False # Make false when there is more behaviour session
alignSubtract = True
#ind = 49
for ind, recordingDate in enumerate(info.recordingList.recordingDate):
#if ind ==49:
    filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
    e_filenameCSV = [f for f in glob.glob(filenameCSV)]
    if len(e_filenameCSV)==1:
        info.recordingList.loc[ind,'eventTimesExtracted']=1
        info.recordingList.loc[ind,'eventTimesPath'] = filenameCSV
    else:
        if checkOnly:
            info.recordingList.loc[ind,'eventTimesExtracted']=0
            info.recordingList.loc[ind,'eventTimesPath'] = filenameCSV
        else:
            try:
                filenameTimeline = [f for f in glob.glob(info.recordingList.filepathname[ind]+ '\\' + info.recordingList.sessionName[ind] + '_Timeline.mat')]
                if  (len(filenameTimeline)>0):
                    sessionProfile ='Grating2AFC'
                else:
                    sessionProfile ='Grating2AFC_noTimeline'
                
                # Get behaviour trial data from Block.mat file
                print(' Extracting time events--> Profile: ' + sessionProfile  +'  Session: ' + info.recordingList.sessionName[ind])
                data = eng.getBehavData(info.recordingList.sessionName[ind],sessionProfile)

                # Apply correction based on the weights to match the eventTimes
                if (alignSubtract) & (len(filenameTimeline)>0) & (info.recordingList.twoP[ind]==True): 
                    print(' Aligning time events: ' + info.recordingList.sessionName[ind])
                    # Get weights to convert from probe to behavioural timebase
                    twoPpath = info.recordingList.path[ind] + '\\TwoP'
                    sessionName = info.recordingList.sessionName[ind]
                    figsavepath = info.recordingList.analysispathname[ind]
                    dataCorrected, variance = eng.applySubtractionCorrection (data, twoPpath ,sessionName, True, figsavepath, nargout=2)
                    info.recordingList.loc[ind,'variance'] = variance

                    # Apply correction to the signal
                    data = dataCorrected
                    
                # Save the file
                eng.writetable(data, filenameCSV, nargout=0)
                info.recordingList.loc[ind,'eventTimesExtracted']=1
            except:
                print(str(ind) + ' - FAILED: Extracting time events: ' + info.recordingList.sessionName[ind])
                info.recordingList.loc[ind,'eventTimesExtracted']=0

# display the output
print( "Behaviour trial data extraction completed: " + 
      str(info.recordingList.eventTimesExtracted.sum()) +"/" + str(info.recordingList.shape[0]))

Behaviour trial data extraction completed: 1.0/1


In [19]:
# default settings for Suite2p analysis
ops = {
        'batch_size': 200, # reduce if running out of RAM
        'fast_disk': os.path.expanduser('~/suite2p_binaries'), # used to store temporary binary file, defaults to save_path0 (set as a string NOT a list)
         #'save_path0': '/media/jamesrowland/DATA/plab/suite_2p', # stores results, defaults to first item in data_path
        'delete_bin': False, # whether to delete binary file after processing
        # main settings
        'nplanes' : 1, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'diameter': 12, # this is the main parameter for cell detection, 2-dimensional if Y and X are different (e.g. [6 12])
        'tau':  1.26, # this is the main parameter for deconvolution (1.25-1.5 for gcamp6s)
        'fs': 30.,  # sampling rate (total across planes)
        # output settings
        'save_mat': True, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        # parallel settings
        'num_workers': 0, # 0 to select num_cores, -1 to disable parallelism, N to enforce value
        'num_workers_roi': 0, # 0 to select number of planes, -1 to disable parallelism, N to enforce value
        # registration settings
        'do_registration': True, # whether to register data
        'nimg_init': 200, # subsampled frames for finding reference image
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': True, # whether to save registered tiffs
        'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
        # cell detection settings
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'navg_frames_svd': 5000, # max number of binned frames for the SVD
        'nsvd_for_roi': 1000, # max number of SVD components to keep for ROI detection
        'max_iterations': 20, # maximum number of iterations to do cell detection
        'ratio_neuropil': 6., # ratio between neuropil basis size and cell radius
        'ratio_neuropil_to_cell': 3, # minimum ratio between neuropil radius and cell radius
        'tile_factor': 1., # use finer (>1) or coarser (<1) tiles for neuropil estimation during cell detection
        'threshold_scaling': 1., # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'outer_neuropil_radius': np.inf, # maximum neuropil radius
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        # deconvolution settings
        'baseline': 'maximin', # baselining mode
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
      }


In [23]:
# Run Suite2p

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    # Check if twoP is True
    if info.recordingList.twoP[ind]:
        tiff_directory = os.path.join(info.recordingList.path[ind], 'TwoP', f"{info.recordingList.recordingDate[ind]}_t-001")
        suite2p_folder = os.path.join(tiff_directory, 'suite2p')
        e_suite2p_folder = [f for f in glob.glob(suite2p_folder)]
        if len(e_suite2p_folder)==1:
                info.recordingList.loc[ind,'suite2pPath'] = suite2p_folder
                print(f"{info.recordingList.animalID[ind]}{info.recordingList.recordingDate[ind]}: Suite2p is created before")
        else:
            db = { 
                        'data_path':  os.path.join(info.rawPath, info.recordingList.animalID[ind]),
                        'tiff_list': glob.glob(os.path.join(tiff_directory, "*.tif")),
                        'save_folder': suite2p_folder
                        }
            from suite2p.run_s2p import run_s2p
            import time        
            t1 = time.time()
            opsEnd = run_s2p(ops=ops,db=db)
            t2 = time.time()
            print(f"{info.recordingList.animalID[ind]}: Suite2p is created in {t2 - t1} seconds.")
    else:
        print(f"Skipping Suite2p for {info.recordingList.animalID[ind]} - twoP is False")
    
print('Completed')

CMN0012025-04-09: Suite2p is created before
Completed


In [ ]:
#extract suite2p output

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    # Define the output file path
    filenameINFO = os.path.join(info.recordingList.analysispathname[ind], 'imaging-data.pkl')
    
    # Check if file already exists
    if os.path.exists(filenameINFO):
        print(f"File already exists for session {info.recordingList.sessionName[ind]}, skipping extraction")
        print(f"Existing file: {filenameINFO}")
    else:
        try:
            # Read suite2p
            s2p_path = os.path.join(info.recordingList.path[ind], 'TwoP', f"{info.recordingList.recordingDate[ind]}_t-001", 'suite2p', 'plane0')
            if os.path.exists(s2p_path):
                print('Extracting...')
                ops = np.load(os.path.join(s2p_path, 'ops.npy'), allow_pickle=True)
                ops = ops.item()
                FrameNums = ops['frames_per_file']
                filelist = ops['filelist']
                isCell = np.load(os.path.join(s2p_path, 'iscell.npy'), allow_pickle=True)

                # Load the suite2p
                flu_raw_subtracted, spks, stat = utils.s2p_loader(s2p_path)
                flu = utils.dfof2(flu_raw_subtracted)

                # Cut each session & save it in the analysis-session folder
                imaging_data = {
                    "n_frames": FrameNums,
                    "flu": flu,
                    "spks": spks,
                    "stat": stat,
                }
                
                print(f"{filenameINFO} --> # of cells: {flu.shape[0]} # of frames: {flu.shape[1]}")
                with open(filenameINFO, 'wb') as f:
                    pickle.dump(imaging_data, f)
                print(f"Successfully saved imaging data for session {info.recordingList.sessionName[ind]}")
            else:
                print(f"Suite2p path not found: {s2p_path}")
        except Exception as e:
            print(f"Error processing session {info.recordingList.sessionName[ind]}: {str(e)}")

        

Extracting...
loading 537 traces labelled as cells
subtracting neuropil with a coefficient of 0.7
C:/Users/Lak Lab/Documents/Github/sideBiasLateralisation\analysis\2025-04-09_CMN001_1\imaging-data.pkl --> # of cells: 537 # of frames: 26211
Successfully saved imaging data for session 2025-04-09_1_CMN001


In [ ]:
# Check Suite2p extraction

for ind, recordingDate in enumerate(info.recordingList.recordingDate):
    filenameDFF = info.recordingList.analysispathname[ind] + 'imaging-data.pkl'
    e_filenameDFF = [f for f in glob.glob(filenameDFF)]
    if len(e_filenameDFF)>0:
        info.recordingList.loc[ind,'imagingDataExtracted']=1
        info.recordingList.loc[ind,'imagingDataPath'] = filenameDFF
    else:
        info.recordingList.loc[ind,'imagingDataExtracted']=0
        info.recordingList.loc[ind,'imagingDataPath'] = filenameDFF

# display the output
print( "Imaging data extraction completed: " + 
      str(info.recordingList.imagingDataExtracted.sum()) +"/" + str(info.recordingList.shape[0]))

Imaging data extraction completed: 1.0/1


In [33]:
# check imaging-data.pkl

import pickle

# File path
file_path = info.recordingList.analysispathname[ind] + 'imaging-data.pkl'

try:
    # Open and load the pickle file
    with open(file_path, 'rb') as f:
        imaging_data = pickle.load(f)
    
    # Show the keys of the dictionary
    print("Keys available in the file:")
    print(imaging_data.keys())
    
    # Show basic data information
    print("\nData information:")
    print(f"Number of frames: {imaging_data['n_frames']}")
    print(f"Flu shape: {imaging_data['flu'].shape}")
    print(f"Spks shape: {imaging_data['spks'].shape}")
    print(f"Number of elements in stat: {len(imaging_data['stat'])}")
    
except FileNotFoundError:
    print(f"Error: File not found in {file_path}")
except Exception as e:
    print(f"Error opening file: {str(e)}")

Keys available in the file:
dict_keys(['n_frames', 'flu', 'spks', 'stat'])

Data information:
Number of frames: [26211]
Flu shape: (537, 26211)
Spks shape: (537, 26211)
Number of elements in stat: 537


In [34]:
# Save info into the analysis folder
filenameINFO = info.analysisPath + '\\infoForAnalysis.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

# Save table as CSV
recordingList = info.recordingList
recordingList.to_csv( info.analysisPath +'\\recordingList.csv', index=False)

All should be done!!
